In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import LicenseAnalysis as LA
LA.printNoCustomers()

KeyError: 'Contract ARR'

<h2>These customers or this customer pays annually this amount of Support to Oracle for Oracle Database:</h2>

In [ ]:
#import LicenseAnalysis as LA
#global totalSupportPaid
LA.printLicenseCosts()



 ------------------------------  Summary of License Costs  ------------------------------
                              Product Name License type  Quantity Contract ARR Average Discount
                         Active Data Guard          NUP       225       $9,753             -81%
                         Active Data Guard    Processor       182     $155,707             -93%
                      Advanced Compression          NUP       175       $3,020             -92%
                      Advanced Compression    Processor       231     $225,919             -91%
                         Advanced Security    Processor       277     $323,771             -92%
Data Masking Pack for Non Oracle Databases    Processor         5       $6,461             -89%
                        Database In-Memory    Processor         5       $8,008             -93%
        Database Lifecycle Management Pack    Processor        17      $16,121             -92%
                            Database Vault   

<h2>Based on the paid support we can analyse how many licenses you have and what is the size of the infrastrucure you can cover with these licenses. </h2>

In [ ]:
import LicenseAnalysis as LA
global max_cores_supported
LA.printLicenseQuantities()


 ------------------------------  Summary of Licenses  ------------------------------
License type                                NUP Processor Total Intel cores  \
Product Name                                                                  
Oracle Database Enterprise Edition          108     1,200             1,308   
Diagnostics Pack                             44     1,112             1,156   
Tuning Pack                                  44     1,020             1,064   
Partitioning                                 38       970             1,008   
Advanced Security                             0       554               554   
Real Application Clusters                    28       484               512   
Advanced Compression                         14       462               476   
Active Data Guard                            18       364               382   
Database Vault                                0        88                88   
Oracle Audit Vault and Database Firewall     

If the customer has a ULA contract with Oracle, it makes difficult to do the sizing, because the customer has unlimited licenses, so we do not know exactly what is the utilized quantity. 

<h2>This (these) customer(s) currently has the following ULA licenses:</h2>

In [ ]:
import LicenseAnalysis as LA
global customerHasULA

customerHasULA=LA.printULAInfo()


 ------------------------------  ULA Information  ------------------------------
These customers has probably a ULA on these Licenses:
                         Customer Name                       Product Name
Polskie Sieci Elektroenergetyczne S.A. Oracle Database Enterprise Edition
Polskie Sieci Elektroenergetyczne S.A.          Real Application Clusters
Polskie Sieci Elektroenergetyczne S.A.                       Partitioning
Polskie Sieci Elektroenergetyczne S.A.                   Diagnostics Pack
Polskie Sieci Elektroenergetyczne S.A.                        Tuning Pack


If the customer would consoldate all the workloads (excluding the workload which is covered by ULA) which are covered by the existing environment into a single ExaCC or ExaCS environment,

<h2>This would be the required configuration to host the whole workload<h2>

In [ ]:
import LicenseAnalysis as LA
LA.printTargetSizing()


 ------------------------------  Possible Exa target For The complete workload  ------------------------------

Target Sizing for this installation:
Configuration       Number    CPU      Storage	Monthly Cost
Full Rack               2       992     1,538	$86,400
Half Rack               1       248       384	$21,600
Quarter Rack            2       248       384	$21,600
Storage Expansion       5         0       320	$10,800
------------------------------------------------------------
Total:                 10     1,488     2,626	$140,400
Total Annual Infrastructure cost: $1,684,800


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import TCOComparison as TCO
import LicenseAnalysis as LA

max_cores_supported=LA.getMaxCoresSupported()
totalSupportPaid=LA.getTotalSupportPaid()

#totalSupportPaid=licenses[licenses["DB_Option"]>0]["Contract ARR"].sum()
TCO_result=TCO.TCO_comparison(
    cores=max_cores_supported,
    storageTB=max_cores_supported*2,
    annualDBsupport=totalSupportPaid,
    utilization=0.4)
Scenarios = ['On Premise', 'ExaCC', 'ExaCS', 'ExaCC BYOL', 'ExaCS BYOL']
Infrastructure = TCO_result['Infrastructure']
License_support = TCO_result['DB Support']
Universal_credits=TCO_result['UC']
width = 0.9       # the width of the bars: can also be len(x) sequence

#Totals=list(map(lambda a,b,c: a+b+c, Infrastructure, License_support,Universal_credits))
Totals=TCO_result['Total']
Reference='On Premise' # On the chart the savings will be displayed based on this reference

data={'Scenarios':Scenarios,'Infra':Infrastructure,'License_support':License_support,'UC':Universal_credits}
df=pd.DataFrame(data)
df.set_index('Scenarios', inplace=True)
fig=go.Figure(
    data=[
        go.Bar(
            name="Infra",
            x=df.index,
            y=df.Infra,
            offsetgroup=0,
            text=list(map(lambda val: 'Infra \n${:,.0f}K'.format(val/1000) if(val>0) else "",df.Infra)),
            textposition="inside"
        ),
         go.Bar(
            name="License_Support",
            x=df.index,
            y=df.License_support,
            offsetgroup=0,
            text=list(map(lambda val: 'Lic. Support \n${:,.0f}K'.format(val/1000) if(val>0) else "",df.License_support)),
            textposition="inside",
            textfont=dict(
                color="white"
            )
        ),       
        go.Bar(
            name="UC",
            x=df.index,
            y=df.UC,
            offsetgroup=0,
            text=list(map(lambda val: 'UC \n${:,.0f}K'.format(val/1000) if(val>0) else "",df.UC)),
            textposition="inside"
        )
    ]
)
fig.update_layout(barmode="stack",bargap=0.1)

totalSum=df.sum(axis="columns")
totalSumText=list(map(lambda val: '{:+.0%} Total \n${:,.0f}K'.format(val/totalSum[Reference]-1,val/1000) if(val>0) else "",totalSum))
fig.add_trace(go.Scatter(
#    x=df.Scenarios, 
    x=df.index,
    y=totalSum,
    text=totalSumText,
    mode="text",
    textposition='top center',
    textfont=dict(
        size=18,
    ),
    showlegend=False
))
fig.update_yaxes(range=[0,totalSum.max()*1.2])
fig.update_layout(title="4 years cost comparison",width=df.index.size*220)
fig.show()